# Shared Data Source Synchronization Issue

1. Import all application;
2. Process different files;
3. Separate files of forms;
4. Restruct application, setting one folder for each form;

1. Process all application Data Modules;
2. Create the Data Module of each form;
3. Cross DMs used by form with application DMs;
3. Code form Data Module;

In [262]:
from pathlib import Path
import pandas as pd

projectSourcePath = Path.cwd()
appFolders        = 'erp-dktp-folders'
originalApp       = 'original/app'
originalAppPath   = Path(projectSourcePath.joinpath('../..', appFolders, originalApp))

appContent     = []
appFolders     = []
appFilesInRoot = []
if originalAppPath.exists() and originalAppPath.is_dir():
   for item in Path.iterdir(originalAppPath):
      name      = str(item).split('/').pop()
      extension = item.suffix if item.suffix != '' else None
      baseName  = name.replace(extension, '') if extension is not None else name
      
      # 

      # files in root
      if not Path(item).is_dir():
         appFilesInRoot.append({
            'baseName': baseName,
            'extension': extension,
            'name': name,
            'path': item
         })
      # folders in root
      else:
         appFolders.append({
            'baseName': baseName,
            'name': name,
            'path': item
         })
      # all content in root
      appContent.append({
         'baseName': baseName,
         'extension': extension,
         'name': name,
         'path': item
      })

In [ ]:
print('\nApp Content In Root Directory\n')
appContentDF = pd.DataFrame(appContent).sort_values('name')
appContentDF.info()
display(appContentDF)

In [ ]:
print('\nApp Folders In Root Directory\n')
appFoldersDF = pd.DataFrame(appFolders).sort_values('name')
appFoldersDF.info()
display(appFoldersDF)

In [ ]:
print('\nApp Files In Root Directory\n')
appFilesInRootDF = pd.DataFrame(appFilesInRoot).sort_values('name')
appFilesInRootDF.info()
display(appFilesInRootDF)
print('Files Without Extension')
display(appFilesInRootDF[appFilesInRootDF['extension'].isna()])

In [ ]:
print('\nApp File Extensions In Root Directory\n')
diffFilesSuffixes = (appFilesInRootDF
                     .groupby('extension')
                     .all()
                     .drop(['baseName', 'name', 'path'], axis=1)
                     .reset_index())
diffFilesSuffixes.info()
display(diffFilesSuffixes)

In [ ]:
print('\nApp Unit Files In Root Directory\n')
unitExtensions     = ['.pas', '.dfm', '.ddp', '.dcu']
unitTempExtensions = ['.~pas', '.~dfm', '.~ddp']

delphiUnitFilesDF = appFilesInRootDF[appFilesInRootDF['extension'].isin(unitExtensions + unitTempExtensions)]
delphiUnitFilesDF.info()
display(delphiUnitFilesDF)

In [268]:
delphiUnitsPasDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.pas'].copy() # pas code files
delphiUnitsDfmDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.dfm'].copy() # dfm files
delphiUnitsDdpDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.ddp'].copy() # ddp files
delphiUnitsDcuDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.dcu'].copy() # dcu files

delphiUnitsTempPasDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.~pas'].copy() # temp pas files
delphiUnitsTempDfmDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.~dfm'].copy() # temp dfm files
delphiUnitsTempDdpDF = delphiUnitFilesDF[delphiUnitFilesDF['extension'] == '.~ddp'].copy() # temp ddp files

In [ ]:
print('\nApp .pas Files in Root Directory\n')
delphiUnitsPasDF.info()

In [ ]:
print('\nApp .dfm Files in Root Directory\n')
delphiUnitsDfmDF.info()

In [ ]:
print('\nApp .ddp Files in Root Directory\n')
delphiUnitsDdpDF.info()

In [ ]:
print('\nApp .dcu Files in Root Directory\n')
delphiUnitsDcuDF.info()

In [ ]:
print('\nApp .~pas Files in Root Directory\n')
delphiUnitsTempPasDF.info()

In [ ]:
print('\nApp .~dfm Files in Root Directory\n')
delphiUnitsTempDfmDF.info()

In [ ]:
print('\nApp .~ddp Files in Root Directory\n')
delphiUnitsTempDdpDF.info()

In [ ]:
delphiUnitsDF = delphiUnitsPasDF

delphiUnitsDF.loc[:, 'dfm']  = None
delphiUnitsDF.loc[:, 'ddp']  = None
delphiUnitsDF.loc[:, 'dcu']  = None
delphiUnitsDF.loc[:, '~pas'] = None
delphiUnitsDF.loc[:, '~dfm'] = None
delphiUnitsDF.loc[:, '~ddp'] = None

delphiUnitsDF = delphiUnitsDF[['baseName', 'dfm', 'ddp', 'dcu', '~pas', '~dfm', '~ddp', 'name', 'extension', 'path']]


# link all unit related files
for index, row in delphiUnitsDF.iterrows():
   sameNameItem = delphiUnitFilesDF.loc[delphiUnitFilesDF['baseName'].str.upper() == str(row['baseName']).upper()]

   if not sameNameItem.empty:
      dfmItem      = sameNameItem.loc[sameNameItem['extension'] == '.dfm', 'name']
      ddpItem      = sameNameItem.loc[sameNameItem['extension'] == '.ddp', 'name']
      dcuItem      = sameNameItem.loc[sameNameItem['extension'] == '.dcu', 'name']
      temp_pasItem = sameNameItem.loc[sameNameItem['extension'] == '.~pas', 'name']
      temp_dfmItem = sameNameItem.loc[sameNameItem['extension'] == '.~dfm', 'name']
      temp_ddpItem = sameNameItem.loc[sameNameItem['extension'] == '.~ddp', 'name']

      if not dfmItem.empty:      delphiUnitsDF.loc[index, 'dfm']  = dfmItem.iloc[0]
      if not ddpItem.empty:      delphiUnitsDF.loc[index, 'ddp']  = ddpItem.iloc[0]
      if not dcuItem.empty:      delphiUnitsDF.loc[index, 'dcu']  = dcuItem.iloc[0]
      if not temp_pasItem.empty: delphiUnitsDF.loc[index, '~pas'] = temp_pasItem.iloc[0]
      if not temp_dfmItem.empty: delphiUnitsDF.loc[index, '~dfm'] = temp_dfmItem.iloc[0]
      if not temp_ddpItem.empty: delphiUnitsDF.loc[index, '~ddp'] = temp_ddpItem.iloc[0]

delphiUnitsDF.info()
display(delphiUnitsDF)

In [ ]:
# dfm files not linked to a pascal file
print('\nDFM Files Not Linked to Pascal File\n')
for index, row in delphiUnitsDfmDF.iterrows():
   item = delphiUnitsDF.loc[delphiUnitsDF['dfm'] == row['name']]
   if item.empty:
      print(row['name'])

# ddp files not linked to a pascal file
print('\nDDP Files Not Linked to Pascal File\n')
for index, row in delphiUnitsDdpDF.iterrows():
   item = delphiUnitsDF.loc[delphiUnitsDF['ddp'] == row['name']]
   if item.empty:
      print(row['name'])

# dcu files not linked to a pascal file
print('\nDCU Files Not Linked to Pascal File\n')
for index, row in delphiUnitsDcuDF.iterrows():
   item = delphiUnitsDF.loc[delphiUnitsDF['dcu'] == row['name']]
   if item.empty:
      print(row['name'])

# ~pas files not linked to a pascal file
print('\n~PAS Files Not Linked to Pascal File\n')
for index, row in delphiUnitsTempPasDF.iterrows():
   item = delphiUnitsDF.loc[delphiUnitsDF['~pas'] == row['name']]
   if item.empty:
      print(row['name'])

# ~dfm files not linked to a pascal file
print('\n~DFM Files Not Linked to Pascal File\n')
for index, row in delphiUnitsTempDfmDF.iterrows():
   item = delphiUnitsDF.loc[delphiUnitsDF['~dfm'] == row['name']]
   if item.empty:
      print(row['name'])

# ~ddp files not linked to a pascal file
print('\n~DDP Files Not Linked to Pascal File\n')
for index, row in delphiUnitsTempDdpDF.iterrows():
   item = delphiUnitsDF.loc[delphiUnitsDF['~ddp'] == row['name']]
   if item.empty:
      print(row['name'])